# Verschlüsselung und Entschlüsselung von Dateien

Im folgenden wird gezeigt, wie die Datei `file_encryption.py` genutzt werden kann, um Dateien zu verschlüsseln und zu entschlüsseln.

Dazu wird asymmetrische Verschlüsselung benutzt. Das ist ein Verfahren, bei dem beide Beteiligte an einer Kommunikation jeweils einen privaten Schlüssel besitzen, den sie geheim halten müssen. Zu jedem privaten Schlüssel gehört ein öffentlicher Schlüssel. 

Diese öffentlichen Schlüssel müssen nicht geheim gehalten werden, sondern können an jeden beliebigen Kommunikationspartner verteilt werden. 

Für eine geheime Kommunikation zwischen den Partnern 1 und 2 gibt es dann zwei Schlüsselpaare

`s_1,p_2`

sowie

`s_2,p_1`

Dabei steht `s` für secret und `p` für public.

Daten, die mit dem Schlüsselpaar `s_1,p_2` verschlüsselt werden, können mit dem Schlüsselpaar `s_2,p_1` wieder entschlüsselt werden. Umgekehrt werden Daten, die mit dem Schlüsselpaar `s_2,p_1` verschlüsselt wurden mit dem Schlüsselpaar `s_1,p_2` wieder entschlüsselt.

Das bedeutet, jeder Kommunikationspartner muss sich nur ein Geheimnis merken, nämlich seinen privaten Schlüssel `s`. Den daraus resultierenden öffentlichen Schlüssel `p` muss er mit allen Kommunikationspartnern teilen. Das kann zum Beispiel durch Veröffentlichung auf der Homepage, auf einer Visitenkarte, per Email, SMS oder mit einem beliebigen Messenger geschehen.

Nach dem Austausch der öffentlichen Schlüssel sollte noch einmal verifiziert werden, dass der öffentliche Schlüssel tatsächlich der richtigen Person zugeordnet wird.

Im folgenden wird dies anhand der Kommunikationspartner **Alice** und **Bob** exemplarisch durchgeführt:

In [1]:
from file_encryption import (
    create_keys, 
    read_secret_key, 
    read_public_key, 
    encrypt, 
    decrypt
)

## Schlüsselverzeichnisse anlegen

### Schlüsselverzeichnis für Alice

In [2]:
path = '.alices_keys' # verstecktes Verzeichnis
secret_key = 'secret_key' # Name des geheimen Schlüssels
public_key = secret_key+'.pub' # Name des eigenen öffentlichen Schlüssels

wenn der Pfad `.alice` schon existiert, wird er gelöscht und mit den passenden Berechtigungen neu angelegt.

Diese Berechtigungen erlauben dem Besitzer des Verzeichnisses, dieses zu lesen, zu schreiben und auszuführen. Andere Nutzer dürfen dieses Verzeichnis weder lesen, noch darin schreiben oder es ausführen. Das wird durch die Angabe

`drwx------`

des Ausdrucks von `!ls -hal <Pfad>` dargestellt. Dabei steht `d` für directory, `r` für read, `w` für write und `x` für execute. Die Berechtigungen werden in der Reihenfolge `user|group|others` angegeben. 

Der Ausdruck

`dr--r--r--`

würde also auf ein Verzeichnis hinweisen, dass von allen (user,group,others) gelesen werden darf, aber in das niemand schreiben darf oder es ausführen darf.

Möchte man nur die Berechtigungen abbilden, so läßt man häufig das führende `d` weg, schreibt also einfach `r--r--r--`.

Eine gesetzte Berechtigung wird häufig durch die Binärzahl 1 angezeigt, eine fehlende Berechtigung durch die Binärzahl 0. Demnach wäre die Berechtigung `rwx------` durch die Binärzahl

`0b111000000`

dargestellt, die der Oktalzahl

`0o700`

entspricht.

In [3]:
import shutil
import os

In [4]:
if os.path.exists(path):
    shutil.rmtree(path,ignore_errors=True)
os.mkdir(path,mode=0o700)

In [5]:
create_keys(os.path.join(path,secret_key))

In [6]:
!ls -hal $path

insgesamt 16K
drwx------ 2 adminloc adminloc 4,0K Aug  1 15:23 .
drwxr-xr-x 8 adminloc adminloc 4,0K Aug  1 15:23 ..
-rw------- 1 adminloc adminloc   64 Aug  1 15:23 secret_key
-rw-r--r-- 1 adminloc adminloc   64 Aug  1 15:23 secret_key.pub


### Schlüsselverzeichnis für Bob

In [7]:
path = '.bobs_keys'
secret_key = 'secret_key'
public_key = secret_key+'.pub'

In [8]:
if os.path.exists(path):
    shutil.rmtree(path,ignore_errors=True)
os.mkdir(path,mode=0o700)

In [9]:
create_keys(os.path.join(path,secret_key))

In [10]:
!ls -hal $path

insgesamt 16K
drwx------ 2 adminloc adminloc 4,0K Aug  1 15:24 .
drwxr-xr-x 8 adminloc adminloc 4,0K Aug  1 15:24 ..
-rw------- 1 adminloc adminloc   64 Aug  1 15:24 secret_key
-rw-r--r-- 1 adminloc adminloc   64 Aug  1 15:24 secret_key.pub


Obwohl die Dateinamen in den Verzeichnissen `.alice` und `.bob` gleich aussehen, ist ihr Inhalt unterschiedlich.

Da die geheimen Schlüssel geheim gehalten werden sollen, wird das nur an den öffentlichen Schlüsseln gezeigt. Öffentliche Schlüssel enden gewöhnlich mit dem Kürzel `.pub`.

In [11]:
!cat .alices_keys/secret_key.pub

03a4b5800e0b62bfe32d95a10a12ddebb59162c0cdddc093c7388e3802de1065

In [12]:
!cat .bobs_keys/secret_key.pub

392072c4dca6714c143e1c5b18433a764642a8578e4b698a1dd27d4e2e22815c

### Schlüsselaustausch von Alice und Bob:

Alice und Bob müssen ihre öffentlichen Schlüssel (die auf `.pub` enden) austauschen. Dabei benennt Bob sinnvollerweise den öffentlichen Schlüssel von Alice mit `alice.pub`, und Alice nennt den öffentlichen Schlüssel von Bob `bob.pub`.

Durch diese eindeutige Kennzeichnung können Alice und Bob sofort ihre privaten und öffentlichen Schlüssel richtig zuordnen.

In [13]:
!cp .alices_keys/secret_key.pub .bobs_keys/alice.pub

In [14]:
!cp .bobs_keys/secret_key.pub .alices_keys/bob.pub

In [15]:
!ls -hal .alices_keys .bobs_keys

.alices_keys:
insgesamt 20K
drwx------ 2 adminloc adminloc 4,0K Aug  1 15:24 .
drwxr-xr-x 8 adminloc adminloc 4,0K Aug  1 15:24 ..
-rw-r--r-- 1 adminloc adminloc   64 Aug  1 15:24 bob.pub
-rw------- 1 adminloc adminloc   64 Aug  1 15:23 secret_key
-rw-r--r-- 1 adminloc adminloc   64 Aug  1 15:23 secret_key.pub

.bobs_keys:
insgesamt 20K
drwx------ 2 adminloc adminloc 4,0K Aug  1 15:24 .
drwxr-xr-x 8 adminloc adminloc 4,0K Aug  1 15:24 ..
-rw-r--r-- 1 adminloc adminloc   64 Aug  1 15:24 alice.pub
-rw------- 1 adminloc adminloc   64 Aug  1 15:24 secret_key
-rw-r--r-- 1 adminloc adminloc   64 Aug  1 15:24 secret_key.pub


### Eine Datei erstellen:

Im folgenden wird eine kurze Datei erstellt, die Alice an Bob verschlüsselt übermitteln will.

In [16]:
with open('alices_text.txt', 'w') as f:
    f.writelines(
        ['Die erste Zeile\n',
         'Die zweite Zeile\n',
         'Ein paar Sonderzeichen: ÄÖÜ äöü ß[{|}]\n']
    )

Zur Kontrolle wird die Datei ausgegeben:

In [17]:
!cat alices_text.txt

Die erste Zeile
Die zweite Zeile
Ein paar Sonderzeichen: ÄÖÜ äöü ß[{|}]


### Verschlüsselung der Datei von Alice für Bob

Um die Datei zu verschlüsseln, benötigt Alice ihren privaten Schlüssel und Bobs öffentlichen Schlüssel. Damit kann sie die Funktion 

`encrypt(secret_key, public_key, input, output)` 

aufrufen. 

Die Datei `input` soll verschlüsselt werden. 

Die Datei `output` endet entweder auf `'.encr'` oder auf `'.encr.gz'` und stellt das verschlüsselte Ergebnis dar. Bei der letzten Endung (`'.encr.gz'`) wird die Datei nicht nur verschlüsselt, sondern zusätzlich mit `gzip` komprimiert. Der Grund ist zum einen die Verkleinerung der Datei, zum anderen kennen Emailprogramme den Mime-Typ `gz`, können diese Datei also einem Dateityp zuordnen.

In [18]:
sk = read_secret_key('.alices_keys/secret_key')
pk = read_public_key('.alices_keys/bob.pub')
encrypt(sk,pk,'alices_text.txt','alices_text.txt.encr.gz')
!ls -hal alices_text*

-rw-r--r-- 1 adminloc adminloc  79 Aug  1 15:24 alices_text.txt
-rw-r--r-- 1 adminloc adminloc 163 Aug  1 15:25 alices_text.txt.encr.gz


Alice kann nun die Verschlüsselte Datei -z.B. per Email- an Bob senden, der sie dann entschlüsselt um sie wieder lesbar zu machen.

### Entschlüsselung der Datei durch Bob

Bob muss wissen, dass die Datei von Alice kommt. Denn zur Entschlüsselung benötigt er seinen eigenen privaten Schlüssel und den öffentlichen Schlüssel von Alice. Sendet Alice die Datei z.B. per Email, sollte klar sein, dass diese von ihr verschlüsselt wurde. 

Mit seinem privaten Schlüssel und Alices öffentlichen Schlüssel kann Bob die Funktion

`decrypt(secret_key,public_key,input,output)`

aufrufen. Dabei hat die Datei `input` entweder die Endung `'.encr'` oder `'.encr.gz'`. 

Der Name `output` der Ausgabedatei kann frei gewählt werden. In der Regel entsteht er dadurch, dass die Endung weggelassen wird. 

So würde aus der Eingabedatei

`input = 'a.txt.encr.gz'`

die Ausgabedatei

`output = 'a.txt'`

werden. Es kann aber auch ein ganz anderer Name verwendet werden, z.B.

`output = 'a_von_alice_<datum_des_heutigen_Tages>.txt'`

Es empfiehlt sich in der Regel die ursprüngliche Endung (hier `'.txt'`) beizubehalten.

In [19]:
sk = read_secret_key('.bobs_keys/secret_key')
pk = read_public_key('.bobs_keys/alice.pub')
decrypt(sk,pk,'alices_text.txt.encr.gz','bobs_encrypted_result.txt')
!ls -hal *.txt

-rw-r--r-- 1 adminloc adminloc 79 Aug  1 15:21 alices_entschluesselter_text.txt
-rw-r--r-- 1 adminloc adminloc 79 Aug  1 15:24 alices_text.txt
-rw-r--r-- 1 adminloc adminloc 79 Aug  1 15:25 bobs_encrypted_result.txt
-rw-r--r-- 1 adminloc adminloc 79 Aug  1 15:17 bobs_nachricht_an_alice.txt
-rw-r--r-- 1 adminloc adminloc 30 Aug  1 15:08 bobs_nachricht.txt
-rw-r--r-- 1 adminloc adminloc 30 Aug  1 15:14 malories_result.txt


In [20]:
!cat bobs_encrypted_result.txt

Die erste Zeile
Die zweite Zeile
Ein paar Sonderzeichen: ÄÖÜ äöü ß[{|}]


### Der Man in the Middle Angriff

Weiter oben wurde empfohlen, den Besitzer des öffentlichen Schlüssels zu verifizieren. Wurde der öffentliche Schlüssel im direkten Kontakt ausgetauscht, so kann man davon ausgehen, dass er korrekt ist.

Wird der öffentliche Schlüssel aber per Email, Messenger oder auf irgendeine andere indirekte Weise ausgetauscht, so könnte ein Fälscher (Person 3, mit den Schlüsseln `s_3,p_3`) sich 'in die Mitte' schleichen. Nennen wir diesen dritten Kommunikationspartner Malori. Wenn es Malori gelingt, sich bei Alice als Bob auszugeben und sich bei Bob als Alice auszugeben, dann kann die Kommunikation so ablaufen:

`Alice(sk_alice,pk_malori) <==> Malori(sk_malori,pk_alice)`

`<=Klartext=>`

`Malori(sk_malori,pk_bob)<==> Bob(sk_bob,pk_malori)`


Die Nachricht von Alice kommt über Malori -verändert oder unverändert- bei Bob an. Weder Alice noch Bob können den Angriff bemerken, wenn sie einmal falsche öffentliche Schlüssel akzeptiert haben.

Malori ist in der Lage, die Kommunikation zwischen Alice und Bob im Klartext mitzulesen und bei Bedarf zu verfälschen. Dies bezeichnet man als Man in the Middle Angriff.

### Schlüsselverzeichnis für Malori

In [21]:
path = '.malories_keys'
secret_key = 'secret_key'
public_key = secret_key+'.pub'

In [22]:
if os.path.exists(path):
    shutil.rmtree(path,ignore_errors=True)
os.mkdir(path,mode=0o700)

In [23]:
create_keys(os.path.join(path,secret_key))

In [24]:
!ls -hal $path

insgesamt 16K
drwx------ 2 adminloc adminloc 4,0K Aug  1 15:25 .
drwxr-xr-x 8 adminloc adminloc 4,0K Aug  1 15:25 ..
-rw------- 1 adminloc adminloc   64 Aug  1 15:25 secret_key
-rw-r--r-- 1 adminloc adminloc   64 Aug  1 15:25 secret_key.pub


### Malori hintergeht Alice und Bob:

Dazu gibt er sich bei Alice als Bob aus und bei Bob als Alice:

In [25]:
!cp .alices_keys/secret_key.pub .malories_keys/alice.pub
!cp .bobs_keys/secret_key.pub .malories_keys/bob.pub

In [26]:
!cp .malories_keys/secret_key.pub .alices_keys/bob.pub
!cp .malories_keys/secret_key.pub .bobs_keys/alice.pub

Bobs Nachricht an Alice:

In [27]:
with open('bobs_nachricht.txt','w') as f:
    f.writelines(['Heirate mich.\n','Ich liebe Dich!\n'])

In [28]:
sk = read_secret_key('.bobs_keys/secret_key')
pk = read_public_key('.bobs_keys/alice.pub')
encrypt(sk,pk,'bobs_nachricht.txt','bobs_nachricht.txt.encr')

Malori entschlüsselt die Nachricht von Bob und liest sie:

In [29]:
sk = read_secret_key('.malories_keys/secret_key')
pk = read_public_key('.malories_keys/bob.pub')
decrypt(sk,pk,'bobs_nachricht.txt.encr','malories_result.txt')
!cat 'malories_result.txt'

Heirate mich.
Ich liebe Dich!


Es kommt, wie es kommen muss:

In [30]:
with open('bobs_nachricht_an_alice.txt', 'w') as f:
    f.writelines(
        [
            'Ich weiß alles über Dich!\n',
            'Ich hasse Dich\n',
            'Ich will nichts mehr von Dir wissen!'
        ]
    )

In [31]:
pk = read_public_key('.malories_keys/alice.pub')
encrypt(sk,pk,'bobs_nachricht_an_alice.txt','bobs_nachricht_an_alice.txt.encr')

Alice entschlüsselt den von Malori verfälschten Text und liest ihn:

In [32]:
sk = read_secret_key('.alices_keys/secret_key')
pk = read_public_key('.alices_keys/bob.pub')
decrypt(sk,pk,'bobs_nachricht_an_alice.txt.encr','alices_entschluesselter_text.txt')
!cat alices_entschluesselter_text.txt

Ich weiß alles über Dich!
Ich hasse Dich
Ich will nichts mehr von Dir wissen!